In [3]:
import geopandas as gpd
import pandas as pd
import py7zr, tempfile
import fiona
import shapely.geometry as sgeo
from tqdm.notebook import tqdm
import glob

In [5]:
network_path = "../../resources/network"
area_path = "../../results/spatial/iris.parquet"

output_path = "../../results/parking/network.parquet"

In [10]:
df_area = gpd.read_parquet(area_path)

In [9]:
df_geometry = []

for archive_path in glob.glob("{}/*".format(network_path)):
    with tempfile.TemporaryDirectory() as directory:
        with py7zr.SevenZipFile(archive_path) as archive:
                print("Extracting", archive_path)
                archive.extractall(directory) 

                source_path = list(glob.glob("{}/**/*.gpkg".format(directory), recursive = True))
                assert len(source_path) == 1
                source_path = source_path[0]

                print("  Processing", source_path)

                with fiona.open(source_path, layer = "troncon_de_route") as source:
                    for record in tqdm(source):
                        if pd.to_numeric(record["properties"]["importance"]) >= 3:
                            df_geometry.append(sgeo.LineString(record["geometry"]["coordinates"]))            

df_geometry = gpd.GeoDataFrame(pd.DataFrame({ "geometry": df_geometry }), crs = "EPSG:2154")

Extracting ../../resources/network/BDTOPO_3-0_TOUSTHEMES_GPKG_LAMB93_D077_2022-03-15.7z
  Processing /tmp/tmpowckvou8/BDTOPO_3-0_TOUSTHEMES_GPKG_LAMB93_D077_2022-03-15/BDTOPO/1_DONNEES_LIVRAISON_2022-03-00088/BDT_3-0_GPKG_LAMB93_D077-ED2022-03-15/BDT_3-0_GPKG_LAMB93_D077-ED2022-03-15.gpkg


  0%|          | 0/362351 [00:00<?, ?it/s]

Extracting ../../resources/network/BDTOPO_3-0_TOUSTHEMES_GPKG_LAMB93_D075_2022-03-15.7z
  Processing /tmp/tmplc4z5r58/BDTOPO_3-0_TOUSTHEMES_GPKG_LAMB93_D075_2022-03-15/BDTOPO/1_DONNEES_LIVRAISON_2022-03-00088/BDT_3-0_GPKG_LAMB93_D075-ED2022-03-15/BDT_3-0_GPKG_LAMB93_D075-ED2022-03-15.gpkg


  0%|          | 0/138559 [00:00<?, ?it/s]

Extracting ../../resources/network/BDTOPO_3-0_TOUSTHEMES_GPKG_LAMB93_D092_2022-03-15.7z
  Processing /tmp/tmpzy9ty12z/BDTOPO_3-0_TOUSTHEMES_GPKG_LAMB93_D092_2022-03-15/BDTOPO/1_DONNEES_LIVRAISON_2022-03-00088/BDT_3-0_GPKG_LAMB93_D092-ED2022-03-15/BDT_3-0_GPKG_LAMB93_D092-ED2022-03-15.gpkg


...

  0%|          | 0/166395 [00:00<?, ?it/s]

Extracting ../../resources/network/BDTOPO_3-0_TOUSTHEMES_GPKG_LAMB93_D078_2022-03-15.7z
  Processing /tmp/tmpl1oi2ljj/BDTOPO_3-0_TOUSTHEMES_GPKG_LAMB93_D078_2022-03-15/BDTOPO/1_DONNEES_LIVRAISON_2022-03-00088/BDT_3-0_GPKG_LAMB93_D078-ED2022-03-15/BDT_3-0_GPKG_LAMB93_D078-ED2022-03-15.gpkg


  0%|          | 0/283329 [00:00<?, ?it/s]

Extracting ../../resources/network/BDTOPO_3-0_TOUSTHEMES_GPKG_LAMB93_D091_2022-03-15.7z
  Processing /tmp/tmpq0x8xfqn/BDTOPO_3-0_TOUSTHEMES_GPKG_LAMB93_D091_2022-03-15/BDTOPO/1_DONNEES_LIVRAISON_2022-03-00088/BDT_3-0_GPKG_LAMB93_D091-ED2022-03-15/BDT_3-0_GPKG_LAMB93_D091-ED2022-03-15.gpkg


  0%|          | 0/232931 [00:00<?, ?it/s]

In [11]:
df_network = []

for index, record in tqdm(df_area.iterrows(), total = len(df_area)):
    indices = df_geometry.sindex.intersection(record["geometry"].bounds)
    df_clip = df_geometry.iloc[indices].clip(record["geometry"])
    df_network.append({ "iris": record["iris"], "length": df_clip["geometry"].length.sum() })
    
df_network = pd.DataFrame.from_records(df_network)

  0%|          | 0/5265 [00:00<?, ?it/s]

In [13]:
df_network.to_parquet(output_path)